# 4. Ensemble_XGBoost
Kaggle score: 

## Import PKGs

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from IPython.display import display

import os
import gc
import time
import zipfile
import h5py
import pickle
import math
from PIL import Image
import shutil

from tqdm import tqdm
from multiprocessing import cpu_count

from sklearn.metrics import confusion_matrix, accuracy_score

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Run names

In [2]:
project_name = 'ic_furniture2018'
step_name = 'Ensemble_XGBoost'
time_str = time.strftime("%Y%m%d_%H%M%S", time.localtime())
run_name = project_name + '_' + step_name + '_' + time_str
print('run_name: ' + run_name)

run_name: ic_furniture2018_Ensemble_XGBoost_20180415_075426


## Project folders

In [3]:
cwd = os.getcwd()
input_folder = os.path.join(cwd, 'input')
output_folder = os.path.join(cwd, 'output')
model_folder = os.path.join(cwd, 'model')
feature_folder = os.path.join(cwd, 'feature')
post_pca_feature_folder = os.path.join(cwd, 'post_pca_feature')
log_folder = os.path.join(cwd, 'log')
print('input_folder: \t\t\t%s' % input_folder)
print('output_folder: \t\t\t%s' % output_folder)
print('model_folder: \t\t\t%s' % model_folder)
print('feature_folder: \t\t%s' % feature_folder)
print('post_pca_feature_folder: \t%s' % post_pca_feature_folder)
print('log_folder: \t\t\t%s' % log_folder)

org_train_folder = os.path.join(input_folder, 'org_train')
org_val_folder = os.path.join(input_folder, 'org_val')
org_test_folder = os.path.join(input_folder, 'org_test')
train_folder = os.path.join(input_folder, 'data_train')
val_folder = os.path.join(input_folder, 'data_val')
test_folder = os.path.join(input_folder, 'data_test')
test_sub_folder = os.path.join(test_folder, 'test')

if not os.path.exists(post_pca_feature_folder):
    os.mkdir(post_pca_feature_folder)
    print('Create folder: %s' % post_pca_feature_folder)

train_json_file = os.path.join(input_folder, 'train.json')
val_json_file = os.path.join(input_folder, 'validation.json')
test_json_file = os.path.join(input_folder, 'test.json')
print('\ntrain_json_file: \t\t%s' % train_json_file)
print('val_json_file: \t\t\t%s' % val_json_file)
print('test_json_file: \t\t%s' % test_json_file)

train_csv_file = os.path.join(input_folder, 'train.csv')
val_csv_file = os.path.join(input_folder, 'validation.csv')
test_csv_file = os.path.join(input_folder, 'test.csv')
print('\ntrain_csv_file: \t\t%s' % train_csv_file)
print('val_csv_file: \t\t\t%s' % val_csv_file)
print('test_csv_file: \t\t\t%s' % test_csv_file)

sample_submission_csv_file = os.path.join(input_folder, 'sample_submission_randomlabel.csv')
print('\nsample_submission_csv_file: \t%s' % sample_submission_csv_file)

input_folder: 			/data1/kaggle/imaterialist-challenge-furniture-2018/input
output_folder: 			/data1/kaggle/imaterialist-challenge-furniture-2018/output
model_folder: 			/data1/kaggle/imaterialist-challenge-furniture-2018/model
feature_folder: 		/data1/kaggle/imaterialist-challenge-furniture-2018/feature
post_pca_feature_folder: 	/data1/kaggle/imaterialist-challenge-furniture-2018/post_pca_feature
log_folder: 			/data1/kaggle/imaterialist-challenge-furniture-2018/log

train_json_file: 		/data1/kaggle/imaterialist-challenge-furniture-2018/input/train.json
val_json_file: 			/data1/kaggle/imaterialist-challenge-furniture-2018/input/validation.json
test_json_file: 		/data1/kaggle/imaterialist-challenge-furniture-2018/input/test.json

train_csv_file: 		/data1/kaggle/imaterialist-challenge-furniture-2018/input/train.csv
val_csv_file: 			/data1/kaggle/imaterialist-challenge-furniture-2018/input/validation.csv
test_csv_file: 			/data1/kaggle/imaterialist-challenge-furniture-2018/input/test.csv


## Preview data

In [4]:
train_csv = pd.read_csv(train_csv_file)
print('train_csv.shape is {0}.'.format(train_csv.shape))
display(train_csv.head(2))

val_csv = pd.read_csv(val_csv_file)
print('val_csv.shape is {0}.'.format(val_csv.shape))
display(val_csv.head(2))

test_csv = pd.read_csv(test_csv_file)
print('test_csv.shape is {0}.'.format(test_csv.shape))
display(test_csv.head(2))

test_csv = pd.read_csv(test_csv_file)
print('test_csv.shape is {0}.'.format(test_csv.shape))
display(test_csv.head(2))

sample_submission_csv = pd.read_csv(sample_submission_csv_file)
print('sample_submission_csv.shape is {0}.'.format(sample_submission_csv.shape))
display(sample_submission_csv.head(2))

train_csv.shape is (194828, 3).


,image_id,label_id,url
0,1,5,https://img13.360buyimg.com/imgzone/jfs/t2857/...
1,2,5,http://www.tengdakeli.cn/350/timg01/uploaded/i...


val_csv.shape is (6400, 3).


,image_id,label_id,url
0,1,38,http://www.ghs.net/public/images/fb/3d/51/3beb...
1,2,63,https://img.alicdn.com/imgextra/TB2chFei9YH8KJ...


test_csv.shape is (12800, 2).


,image_id,url
0,1,https://img13.360buyimg.com/imgzone/jfs/t13174...
1,2,http://img35.ddimg.cn/79/22/1258168705-1_u.jpg


test_csv.shape is (12800, 2).


,image_id,url
0,1,https://img13.360buyimg.com/imgzone/jfs/t13174...
1,2,http://img35.ddimg.cn/79/22/1258168705-1_u.jpg


sample_submission_csv.shape is (12800, 2).


,id,predicted
0,1,57
1,2,74


In [5]:
train_id = train_csv['image_id']
train_label_id = train_csv['label_id']

id_2_train_label_id_dict = dict(zip(train_id, train_label_id))
print('len(id_2_train_label_id_dict)=%d' % len(id_2_train_label_id_dict))

index = 0
print('id: %s, \tlandmark_id:%s' % (train_id[index], id_2_train_label_id_dict[train_id[index]]))
index = 1
print('id: %s, \tlandmark_id:%s' % (train_id[index], id_2_train_label_id_dict[train_id[index]]))

image_file = '%s_%s.jpg' % (train_id[index], id_2_train_label_id_dict[train_id[index]])
print(image_file)

len(id_2_train_label_id_dict)=194828
id: 1, 	landmark_id:5
id: 2, 	landmark_id:5
2_5.jpg


In [6]:
val_id = val_csv['image_id']
val_label_id = val_csv['label_id']

id_2_val_label_id_dict = dict(zip(val_id, val_label_id))
print('len(id_2_val_label_id_dict)=%d' % len(id_2_val_label_id_dict))

index = 0
print('id: %s, \tlandmark_id:%s' % (val_id[index], id_2_val_label_id_dict[val_id[index]]))
index = 1
print('id: %s, \tlandmark_id:%s' % (val_id[index], id_2_val_label_id_dict[val_id[index]]))

image_file = '%s_%s.jpg' % (val_id[index], id_2_val_label_id_dict[val_id[index]])
print(image_file)

len(id_2_val_label_id_dict)=6400
id: 1, 	landmark_id:38
id: 2, 	landmark_id:63
2_63.jpg


In [7]:
test_id = test_csv['image_id']

index = 0
print('id: %s' % (test_id[index]))
index = 1
print('id: %s' % (test_id[index]))

image_file = '%s.jpg' % (test_id[index])
print(image_file)

id: 1
id: 2
2.jpg


## Load predict probability files

In [8]:
ori_proba_files = [
    ('proba_ic_furniture2018_Train-Predict_Mix3model_20180415_053404_7941.p', np.nan),
    ('proba_ic_furniture2018_Train-Predict_Mix3model_20180415_061203_7987.p', np.nan),
    ('proba_ic_furniture2018_Train-Predict_Mix3model_20180415_070425_8115.p', np.nan),
#     ('ic_furniture2018_Train-Predict_XIRmixed_20180331_102545_8018.h5', 0.20625),
#     ('ic_furniture2018_Train-Predict_XIRmixed_20180331_083915_7678.h5', 0.22005),
#     ('ic_furniture2018_Train-Predict_XIRmixed_20180331_054212_7823.h5', 0.21979)
]

for pair in ori_proba_files:
    if os.path.exists(os.path.join(model_folder, pair[0])):
        print('File exists: %s' % pair[0])
    else:
        print('***File do not exists: %s' % pair[0])

File exists: proba_ic_furniture2018_Train-Predict_Mix3model_20180415_053404_7941.p
File exists: proba_ic_furniture2018_Train-Predict_Mix3model_20180415_061203_7987.p
File exists: proba_ic_furniture2018_Train-Predict_Mix3model_20180415_070425_8115.p


In [9]:
def save_proba(y_train_proba, y_train, y_val_proba, y_val, y_test_proba, test_filenames, file_name):
    test_filenames = [n.encode('utf8') for n in test_filenames]
    print(test_filenames[:10])
    if os.path.exists(file_name):
        os.remove(file_name)
        print('File removed: \t%s' % file_name)
    with h5py.File(file_name) as h:
        h.create_dataset('y_train_proba', data=y_train_proba)
        h.create_dataset('y_train', data=y_train)
        h.create_dataset('y_val_proba', data=y_val_proba)
        h.create_dataset('y_val', data=y_val)
        h.create_dataset('y_test_proba', data=y_test_proba)
        h.create_dataset('test_filenames', data=test_filenames)
    print('File saved: \t%s' % file_name)

def load_proba(file_name):
    with h5py.File(file_name, 'r') as h:
        y_train_proba = np.array(h['y_train_proba'])
        y_train = np.array(h['y_train'])
        y_val_proba = np.array(h['y_val_proba'])
        y_val = np.array(h['y_val'])
        y_test_proba = np.array(h['y_test_proba'])
        test_filenames = np.array(h['test_filenames'])
    print('File loaded: \t%s' % file_name)
    test_filenames = [n.decode('utf8') for n in test_filenames]
    print(test_filenames[:10])
    
    return y_train_proba, y_train, y_val_proba, y_val, y_test_proba, test_filenames

y_train_probas = []
y_trains = []
y_val_probas = []
y_vas = []
y_test_probas = []
for n in ori_proba_files:
    y_proba_file = os.path.join(model_folder, ori_proba_files[0][0])
    print('*'*60)
    y_train_proba, y_train, y_val_proba, y_val, y_test_proba, test_filenames = load_proba(y_proba_file)
    print(y_train_proba.shape)
    print(y_train.shape)
    print(y_val_proba.shape)
    print(y_val.shape)
    print(y_test_proba.shape)
    print(len(test_filenames))
    y_train_probas.append(y_train_proba)
    y_val_probas.append(y_val_proba)
    y_test_probas.append(y_test_proba)

************************************************************
File loaded: 	/data1/kaggle/imaterialist-challenge-furniture-2018/model/proba_ic_furniture2018_Train-Predict_Mix3model_20180415_053404_7941.p
['test/1.jpg', 'test/10.jpg', 'test/100.jpg', 'test/1000.jpg', 'test/10000.jpg', 'test/10001.jpg', 'test/10002.jpg', 'test/10003.jpg', 'test/10004.jpg', 'test/10005.jpg']
(191261, 128)
(191261,)
(6301, 128)
(6301,)
(12652, 128)
12652
************************************************************
File loaded: 	/data1/kaggle/imaterialist-challenge-furniture-2018/model/proba_ic_furniture2018_Train-Predict_Mix3model_20180415_053404_7941.p
['test/1.jpg', 'test/10.jpg', 'test/100.jpg', 'test/1000.jpg', 'test/10000.jpg', 'test/10001.jpg', 'test/10002.jpg', 'test/10003.jpg', 'test/10004.jpg', 'test/10005.jpg']
(191261, 128)
(191261,)
(6301, 128)
(6301,)
(12652, 128)
12652
************************************************************
File loaded: 	/data1/kaggle/imaterialist-challenge-furniture-2018

## Ensemble

In [10]:
import xgboost as xgb

In [11]:
xg_train = xgb.DMatrix(y_train_proba, label=y_train)
xg_val = xgb.DMatrix(y_val_proba, label=y_val)
xg_test = xgb.DMatrix(y_test_proba)

# setup parameters for xgboost
param = {}
# use softmax multi-class classification
# param['objective'] = 'binary:logistic'
param['objective'] = 'multi:softmax'
# scale weight of positive examples
param['eta'] = 0.1
param['max_depth'] = 10
param['silent'] = False
param['nthread'] = 6
param['num_class'] = 128
param['learning_rate'] = 0.1
param['n_estimators'] = 500


watchlist = [(xg_train, 'train'), (xg_val, 'val')]
num_round = 10
bst = xgb.train(param, xg_train, num_round, watchlist)
# get prediction
prob_val = bst.predict(xg_val)
print(prob_val.shape)
print(prob_val[0:5])
pred_val = (prob_val>0.5).astype(int)
print(pred_val[0:5])
score_val = accuracy_score(y_val, pred_val)
print('score:{0}'.format(score_val))

prob_test = bst.predict(xg_test)
print(prob_test.shape)
print(prob_test[0:5])
pred_test = (prob_test>0.5).astype(int)
print(pred_test[0:10])

[0]	train-merror:0.10355	val-merror:0.210125
[1]	train-merror:0.096392	val-merror:0.206475
[2]	train-merror:0.090327	val-merror:0.205999
[3]	train-merror:0.085647	val-merror:0.205206
[4]	train-merror:0.081784	val-merror:0.20584
[5]	train-merror:0.07839	val-merror:0.205523
[6]	train-merror:0.075651	val-merror:0.204888
[7]	train-merror:0.072947	val-merror:0.205206
[8]	train-merror:0.070563	val-merror:0.205682
[9]	train-merror:0.067578	val-merror:0.204888
(6301,)
[ 0. 93.  0.  3.  0.]
[0 1 0 1 0]
score:0.013966037136962386
(12652,)
[ 11. 116.  54.  25.  41.]
[1 1 1 1 1 1 1 1 1 1]


In [18]:
prob_val = bst.predict(xg_val)
print(prob_val.shape)

(6301,)


## Predict

In [12]:
%%time
max_indexes = np.argmax(prob_test, -1)
print(max_indexes.shape)

test_dict = {}
for pair in zip(test_filenames, max_indexes):
    image_name, indx = pair[0], int(pair[1])
    image_name = image_name.split('/')[-1]
    image_id = int(image_name.split('.')[0])
#     print(pair[0], image_name, image_id, indx, indx+1, type(image_id), type(indx))
    test_dict[image_id] = indx + 1

#确认图片的id是否能与ImageDataGenerator()对应上
for name in test_filenames[:10]:
    image_name = name.split('/')[-1]
    image_id = int(image_name.split('.')[0])
#     print('%s\t%s\t%s' % (name, image_id, test_dict[image_id]))

()


TypeError: zip argument #2 must support iteration

In [13]:
display(sample_submission_csv.head(2))

,id,predicted
0,1,57
1,2,74


In [14]:
%%time
len_sample_submission_csv = len(sample_submission_csv)
print('len(len_sample_submission_csv)=%d' % len_sample_submission_csv)
count = 0
for i in range(len_sample_submission_csv):
    image_id = int(sample_submission_csv.iloc[i, 0])
    if image_id in test_dict:
        pred_label = test_dict[image_id]
#         print('%s\t%s' % (image_id, pred_label))
        sample_submission_csv.iloc[i, 1] = pred_label
    else:
#         print('%s\t%s' % (image_id, 20))
        sample_submission_csv.iloc[i, 1] = 20 # 属于20的类最多，所以全都设置成这个类，可能会比设置成其他得到的结果好
    count += 1
    if count % 1000 == 0:
        print(int(count/1000), end=' ')
display(sample_submission_csv.head(2))

len(len_sample_submission_csv)=12800
1 2 3 4 5 6 7 8 9 10 11 12 

,id,predicted
0,1,20
1,2,20


CPU times: user 11.1 s, sys: 752 ms, total: 11.8 s
Wall time: 11.8 s


In [15]:
print(list(set(sample_submission_csv['predicted'])))

[20]


In [16]:
pred_file = os.path.join(output_folder, 'pred_' + run_name + '.csv')
sample_submission_csv.to_csv(pred_file, index=None)

In [17]:
print(run_name)
print('Done !')

ic_furniture2018_Ensemble_XGBoost_20180415_075426
Done !
